In [267]:
import pandas as pd 
import re 
import numpy as np 
import nltk
from string import punctuation
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split 
import pickle
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [236]:
df = pd.read_csv('C:\\Users\\User\Desktop\\Job-recommendation-system-\\backend\\recommender\\resume.csv')
print(df.Category.unique())
important_category = ['Data Science', 'HR', 'Web Designing', 'Java Developer', 'Python Developer', 'Electrical Engineering', 'Database','Network Security Engineer','Testing', 'Business Analyst']
df = df[df.Category.map(lambda x:x in important_category)]


['Data Science' 'HR' 'Advocate' 'Arts' 'Web Designing'
 'Mechanical Engineer' 'Sales' 'Health and fitness' 'Civil Engineer'
 'Java Developer' 'Business Analyst' 'SAP Developer' 'Automation Testing'
 'Electrical Engineering' 'Operations Manager' 'Python Developer'
 'DevOps Engineer' 'Network Security Engineer' 'PMO' 'Database' 'Hadoop'
 'ETL Developer' 'DotNet Developer' 'Blockchain' 'Testing']


In [237]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s0-9]' % re.escape(punctuation), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    resumeText = re.sub(r'\b\w\b', '', resumeText)
    return resumeText
df.Resume = df.Resume.map(cleanResume)

In [238]:
stopwords_list = stopwords.words('english')
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     max_features=5000,
                     stop_words=stopwords_list)

In [239]:
sequences = [resume for resume in df.Resume.values]
labels = [label for label in df.Category.values]
vectorizer.fit(sequences)

TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [240]:
vector_form = vectorizer.transform(sequences)
category = df.Category.unique()
map_index = {catego:index for index, catego in enumerate(category)}
hashmap = {index:catego for index, catego in enumerate(category)}
df['Category'] = df.Category.map(map_index)


In [241]:
y = df.Category.values

In [242]:
x_train, x_test, y_train, y_test = train_test_split(vector_form, y, test_size = 20, random_state=0)
model = RandomForestClassifier()
model.fit(x_train, y_train)
# model.predict(x_train)

RandomForestClassifier()

In [243]:
with open('./saved_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [244]:
with open('./saved_model.pkl', 'rb') as file:
    model = pickle.load(file)
model.predict(x_test)

array([8, 9, 6, 9, 3, 8, 0, 7, 8, 2, 9, 1, 6, 4, 9, 0, 2, 0, 0, 5],
      dtype=int64)

In [245]:
with open('./hashmap.pkl', 'wb') as file:
    pickle.dump(hashmap, file)


In [246]:
with open('./hashmap.pkl', 'rb') as file:
    hashmap = pickle.load(file)
    

In [271]:
with open('./vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)
    

In [247]:
hashmap[0]

'Data Science'

In [248]:
hashmap

{0: 'Data Science',
 1: 'HR',
 2: 'Web Designing',
 3: 'Java Developer',
 4: 'Business Analyst',
 5: 'Electrical Engineering',
 6: 'Python Developer',
 7: 'Network Security Engineer',
 8: 'Database',
 9: 'Testing'}

In [283]:
text = """
Bachelor's Degree in a Computer Science or related technical discipline, preferred but not required 
10+ years of experience with Java, HTML (HTML5), CSS, JavaScript, JSON, PL SQL, Eclipse, (Subversion or GitHub) 
Must be able to obtain a Position of Public Trust Clearance
US Citizen or Permanent Resident (Green Card Holder)
The security clearance for this program requires the selected candidate to have resided in the US for the past five years. The selected candidate cannot have left the country for longer than 90 consecutive days and no more than 180 cumulative days. 
Bachelor's Degree in a Computer Science or related technical discipline, preferred but not required
10+ years of experience with Java, HTML(HTML5), CSS, JavaScript, JSON, PL SQL, Eclipse, Subversion or GitHub
Knowledge/experience with Agile Methodologies
Knowledge/experience of SDLC practices
Change Management and Configuration Management (VersionOne, ServiceNow, etc.)
Application Lifecycle Management (ALM)
Desired experience with Unix, Jenkins, Sonar, AppScan, Continuous Integration and Build Automation
Ability to manage development projects from initial conception to production
Strong oral and written communication skills
Work with all levels of the organization, from leadership and senior management to the workforce; interactions may include coaching, interviewing, workshop facilitation and training
Familiarity with Agile/Scrum processes and management tools such as VersionOne
Knowledge of and experience in an IT Service management System (preferably ServiceNow)
 Location:
Remote
Full-flex work week to own your priorities at work and at home
401K with company match 
Internal mobility team dedicated to helping you own your career 
Professional growth opportunities including paid education and certifications
Cutting-edge technology you can learn from 
Rest and recharge with paid vacation and holidays
#gditcareers #java #agile #alm #html5

The likely salary range for this position is $96,000 - $144,000, this is not, however, a guarantee of compensation or salary; rather, salary will be set based on experience, geographic location and possibly contractual requirements and could fall outside of this range.
"""

In [262]:
text = cleanResume(text)
# text

In [263]:
vec_text = np.array(text).reshape(1,)

In [251]:
# text.shape

In [255]:
# vec_text = vectorizer.transform(vec_text)

In [256]:
model.predict(vec_text)

array([1], dtype=int64)

In [288]:
class Preprocessor:
    def __init__(self, description) -> None:
        self.cleaned_text = np.array(self.cleanResume(description)).reshape(1,)
        self.category = self.predictor()
    def cleanResume(self, resumeText):
        resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
        resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
        resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
        resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
        resumeText = re.sub('[%s0-9]' % re.escape(punctuation), ' ', resumeText)  # remove punctuations
        resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
        resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
        resumeText = re.sub(r'\b\w\b', '', resumeText)
        return resumeText
    def predictor(self):
            
        with open('./vectorizer.pkl', 'rb') as file:
            vectorizer = pickle.load(file)
        with open('./saved_model.pkl', 'rb') as file:
            model = pickle.load(file)
        with open('./hashmap.pkl', 'rb') as file:
            hashmap = pickle.load(file)        
            
        sequence = self.cleaned_text
        vector_form = vectorizer.transform(sequence)
        class_type = model.predict(vector_form)
        return hashmap[class_type[0]]

In [268]:
description = """
What is happening guys here. I don't think that this is the problem for delaying the preprocessing stage""" 

In [289]:
# text = np.array(description).reshape(1, )
preprocessor = Preprocessor(text)
preprocessor.category

'HR'